## Solving bidomain equations on the slice with the Beeler-Reuter model

### Importing required libraries

In [ ]:
%cd ../../..
from src.models.dynamics_models import *
from src.models.cell_models import *
from src.utils import *
import ufl

### Defining a domain, cell model and dynamics model and visualizing initial data

In [ ]:
domain = heart_slice()
x = ufl.SpatialCoordinate(domain)

In [ ]:
class CellModel(BeelerReuter):
    def apply_current(self):
        locator1 = lambda x: (x[0] - 0) ** 2 + (x[1] - 2.1) ** 2 < 0.7**2
        locator2 = lambda x: (x[0] - 1.2) ** 2 + (x[1] - 1.5) ** 2 < 0.7**2
        locator3 = lambda x: (x[0] - 3.3) ** 2 + (x[1] - 3.1) ** 2 < 0.3**2
        cells1 = fem.locate_dofs_geometrical(self.V1, locator1)
        cells2 = fem.locate_dofs_geometrical(self.V1, locator2)
        cells3 = fem.locate_dofs_geometrical(self.V1, locator3)
        self.I_app.x.array[cells1] = np.full_like(cells1, 10)
        self.I_app.x.array[cells2] = np.full_like(cells2, 10)
        self.I_app.x.array[cells3] = np.full_like(cells3, 10)
        self.I_app_duration = 20

In [ ]:
class Model(BidomainModel):
    def initial_V_m(self):
        self.V_m_n.x.array[:] = -85

    def conductivity(self):
        # Muscle fibres
        x_c, y_c = 0.2, 1
        self.fibers = ufl.as_vector(
            [
                -(x[1] - y_c) / ufl.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
                (x[0] - x_c) / ufl.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
            ]
        )

        # Healthy conductivities
        self.M_i = self.SIGMA_IT * ufl.Identity(2) + (
            self.SIGMA_IL - self.SIGMA_IT
        ) * ufl.outer(self.fibers, self.fibers)
        self.M_e = self.SIGMA_ET * ufl.Identity(2) + (
            self.SIGMA_EL - self.SIGMA_ET
        ) * ufl.outer(self.fibers, self.fibers)

In [ ]:
cell_model = CellModel(domain)
model = Model(domain, cell_model)

In [ ]:
cell_model.plot_I_app(
    show_mesh=False,
    function_name="",
    camera_direction="xy",
    cmap="plasma",
    save_to="figures/slice/BR_applied_current.pdf",
)

### Solving equations with given parameters

In [ ]:
model.solve(
    T=400,
    steps=10000,
    save_to="slice/bidomain_BR.mp4",
    signal_point=[1.0, 3.0, 0.0],
    camera_direction="xy",
    checkpoints=[35,85,230,300],
    checkpoint_file='figures/slice/bidomain_BR'
)

### Plotting a fiber orientations

In [ ]:
x_c, y_c = 0.2, 1
plot_vector_field(
    domain,
    lambda x: (
        -(x[1] - y_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        (x[0] - x_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        0,
    ),
    0.03,
    0.7,
    camera_direction='xy',
    save_to='figures/slice/BR_fibers.pdf'
)

### Plotting a final plot of transmembrane potential

In [ ]:
plot_function(model.V_m_n, "V_m", show_mesh=False)

### Plotting a transmembrane potential in a given signal point

In [ ]:
model.plot_signal()